In [ ]:
!pip install cohere

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

cohere_api_key = os.getenv("COHERE_API_KEY")
openai_api_key = os.getenv("OPENAI_API_KEY")
weaviate_cloud_url = os.getenv("WIKI_DEMO_URL")
weaviate_cloud_key = os.getenv("WIKI_DEMO_KEY_READ")

In [ ]:
import cohere
co = cohere.Client(cohere_api_key)

def generate_query_from_promt(prompt):
    response = co.chat(
        message=prompt,
        search_queries_only=True
    )
    
    return response.search_queries[0].text

In [ ]:
# Example of how to generate a query from a prompt
generate_query_from_promt("Where do the tallest penguins live?")

In [ ]:
import weaviate

client = weaviate.connect_to_weaviate_cloud(
    cluster_url=weaviate_cloud_url,
    auth_credentials=weaviate.auth.AuthApiKey(weaviate_cloud_key),

    headers={
        "X-OpenAI-Api-Key": openai_api_key,
        "X-Cohere-Api-Key": cohere_api_key,
    },

    additional_config=weaviate.config.AdditionalConfig(timeout=(500, 1500))
)

client.is_ready()

In [ ]:
def two_step_rag(user_prompt):
    # Step 1
    prompt = user_prompt + " Please only use the provided content with this prompt. Don't make things up."
    
    generated_query = generate_query_from_promt(prompt)
    print("=== Generated Query ===")
    print(f"Generated query: {generated_query}")

    # Step 2
    wiki = client.collections.get("Wikipedia")

    response = wiki.generate.near_text(
        query=generated_query,
        limit=3,
        grouped_task=prompt,
        grouped_properties=["text", "title"]
    )

    # Print results
    print("=== Generated Response ===")
    print(response.generated)

    print("=== Source ===")
    for item in response.objects:
        print(item.properties)

In [ ]:
# two_step_rag("What wild animals do we know about?")
two_step_rag("How do airplanes fly?")

In [ ]:
two_step_rag("What are the pros and cons of automation using computer?")

In [ ]:
two_step_rag("How do CPUs work?")